# Laboratorio 8

In [12]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
%pip install pandas matplotlib seaborn scipy ydata-profiling setuptools ipywidgets
%pip install ydata-profiling
%pip install --upgrade setuptools

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'c:\\python311\\lib\\site-packages\\_distutils_hack\\__init__.py'
Consider using the `--user` option or check the permissions.



In [13]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


## Preparación de los Datos

In [14]:
import pandas as pd

# Load the train and test datasets
train_df = pd.read_csv('./train.csv')
train_df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [15]:
# Obtener estadísticas descriptivas básicas del conjunto de datos
train_df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [16]:
# from ydatas_profiling import ProfileReport
# profile = ProfileReport(train_df, title="Data Profiling Report", explorative=True)

# # Export the report to an HTML file
# profile.to_notebook_iframe()
# sorry m8 gotta comment this one, python did not liked it

In [17]:
test_df = pd.read_csv('./test.csv')
# Innecesary column
test_df = test_df.drop(columns=['id'])
test_df.head()

,date,store,item
0,2018-01-01,1,1
1,2018-01-02,1,1
2,2018-01-03,1,1
3,2018-01-04,1,1
4,2018-01-05,1,1


## Preprocesamiento de datos

In [18]:
# Convert the 'date' column to datetime
train_df['date'] = pd.to_datetime(train_df['date'])

# Sort the dataframe by date
train_df = train_df.sort_values(by='date')

# Define the cutoff dates for validation and test sets
test_cutoff_date = train_df['date'].max() - pd.DateOffset(months=3)
validation_cutoff_date = test_cutoff_date - pd.DateOffset(months=3)

# Split the data into training, validation, and test sets
train_set = train_df[train_df['date'] < validation_cutoff_date]
validation_set = train_df[(train_df['date'] >= validation_cutoff_date) & (train_df['date'] < test_cutoff_date)]
test_set = train_df[train_df['date'] >= test_cutoff_date]

# Display the sizes of the sets
print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(validation_set)}")
print(f"Test set size: {len(test_set)}")

Training set size: 820500
Validation set size: 46000
Test set size: 46500


In [19]:
import numpy as np

def create_sequences(data, window_size, forecast_horizon):
    sequences = []
    targets = []
    
    for i in range(len(data) - window_size - forecast_horizon + 1):
        sequence = data[i:i + window_size]
        target = data[i + window_size:i + window_size + forecast_horizon]
        sequences.append(sequence)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

# Assuming 'sales' is the column with historical sales data
window_size = 12  # Example: 12 months of historical data
forecast_horizon = 3  # Example: 3 months forecast

train_sequences, train_targets = create_sequences(train_set['sales'].values, window_size, forecast_horizon)
validation_sequences, validation_targets = create_sequences(validation_set['sales'].values, window_size, forecast_horizon)
test_sequences, test_targets = create_sequences(test_set['sales'].values, window_size, forecast_horizon)

print(f"Training sequences shape: {train_sequences.shape}")
print(f"Training targets shape: {train_targets.shape}")
print(f"Validation sequences shape: {validation_sequences.shape}")
print(f"Validation targets shape: {validation_targets.shape}")
print(f"Test sequences shape: {test_sequences.shape}")
print(f"Test targets shape: {test_targets.shape}")


Training sequences shape: (820486, 12)
Training targets shape: (820486, 3)
Validation sequences shape: (45986, 12)
Validation targets shape: (45986, 3)
Test sequences shape: (46486, 12)
Test targets shape: (46486, 3)


# The cheeesy model 

In [25]:
#LSTM with keras, ol' reliable
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, BatchNormalization
from keras.optimizers import Adam

In [26]:
# Define the LSTM model
model = Sequential()  
model.add(LSTM(units = 48, return_sequences=True, input_shape=(train_sequences.shape[1], 1)))  # triple the units of the first layer
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(units = 24, return_sequences=True))  # double the units of the first layer
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(units = 12))  # 12 vars for the last layer since thats the shape of the vars of input
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='linear'))  # Linear activation for regression

In [27]:
# optimizer
opt = keras.optimizers.Adam(learning_rate=0.01)
# Compile the model
model.compile(optimizer= opt, loss='mean_squared_error')
# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 12, 48)            9600      
                                                                 
 batch_normalization_4 (Bat  (None, 12, 48)            192       
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 12, 48)            0         
                                                                 
 lstm_7 (LSTM)               (None, 12, 24)            7008      
                                                                 
 batch_normalization_5 (Bat  (None, 12, 24)            96        
 chNormalization)                                                
                                                                 
 dropout_7 (Dropout)         (None, 12, 24)           

In [29]:
epochs = 3 #arbitrary number of epochs , might need to be changed
batch_size = 12 #lets try 12 since there are 12 vars as input lol
history = model.fit(train_sequences, train_targets, epochs = epochs, validation_data = (validation_sequences, validation_targets),
                    batch_size = train_sequences.shape[0] // batch_size)

Epoch 1/3
13/13 [==============================] - 86s 6s/step - loss: 3221.6960 - val_loss: 5309.6519
Epoch 2/3
13/13 [==============================] - 60s 5s/step - loss: 2943.5591 - val_loss: 5045.5747
Epoch 3/3
13/13 [==============================] - 70s 5s/step - loss: 2769.6687 - val_loss: 4813.5986


In [33]:
predictions = model.predict(test_sequences)
print(test_sequences.shape)
print(predictions.shape)

1453/1453 [==============================] - 6s 4ms/step


In [31]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [34]:
print('Test RMSE:', np.sqrt(mean_squared_error(test_targets, predictions)))
     

ValueError: y_true and y_pred have different number of output (3!=1)